***GENERATED CODE FOR schoolpredictionp PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country Name', 'transformation_label': 'String Indexer'}], 'feature': 'Country Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '266', 'mean': '', 'stddev': '', 'min': 'Afghanistan', 'max': 'Zimbabwe', 'missing': '0', 'distinct': '266'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country Name'}, {'feature_label': 'Country Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country Code', 'transformation_label': 'String Indexer'}], 'feature': 'Country Code', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '266', 'mean': '', 'stddev': '', 'min': 'ABW', 'max': 'ZWE', 'missing': '0', 'distinct': '266'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country Code'}, {'feature_label': 'Country Code', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country Code')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Indicator Name', 'transformation_label': 'String Indexer'}], 'feature': 'Indicator Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '266', 'mean': '', 'stddev': '', 'min': 'Children out of school, primary, male', 'max': 'Children out of school, primary, male', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Indicator Name'}, {'feature_label': 'Indicator Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Indicator Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Indicator Code', 'transformation_label': 'String Indexer'}], 'feature': 'Indicator Code', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '266', 'mean': '', 'stddev': '', 'min': 'SE.PRM.UNER.MA', 'max': 'SE.PRM.UNER.MA', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Indicator Code'}, {'feature_label': 'Indicator Code', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Indicator Code')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1960', 'transformation_label': 'String Indexer'}], 'feature': '1960', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1960'}, {'feature_label': '1960', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1960')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1961', 'transformation_label': 'String Indexer'}], 'feature': '1961', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1961'}, {'feature_label': '1961', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1961')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1962', 'transformation_label': 'String Indexer'}], 'feature': '1962', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1962'}, {'feature_label': '1962', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1962')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1963', 'transformation_label': 'String Indexer'}], 'feature': '1963', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1963'}, {'feature_label': '1963', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1963')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1964', 'transformation_label': 'String Indexer'}], 'feature': '1964', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1964'}, {'feature_label': '1964', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1964')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1965', 'transformation_label': 'String Indexer'}], 'feature': '1965', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1965'}, {'feature_label': '1965', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1965')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1966', 'transformation_label': 'String Indexer'}], 'feature': '1966', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1966'}, {'feature_label': '1966', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1966')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1967', 'transformation_label': 'String Indexer'}], 'feature': '1967', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1967'}, {'feature_label': '1967', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1967')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1968', 'transformation_label': 'String Indexer'}], 'feature': '1968', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1968'}, {'feature_label': '1968', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1968')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '1969', 'transformation_label': 'String Indexer'}], 'feature': '1969', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '266', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '1969'}, {'feature_label': '1969', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('1969')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run schoolpredictionpHooks.ipynb
try:
	#sourcePreExecutionHook()

	countrycomparison = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/CountryComparison.csv', 'filename': 'CountryComparison.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(countrycomparison)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run schoolpredictionpHooks.ipynb
try:
	#transformationPreExecutionHook()

	schoolpredictionpautofe = TransformationMain.run(countrycomparison,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Country Name", "transformation_label": "String Indexer"}], "feature": "Country Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "266", "mean": "", "stddev": "", "min": "Afghanistan", "max": "Zimbabwe", "missing": "0", "distinct": "266"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country Name"}, {"transformationsData": [{"feature_label": "Country Code", "transformation_label": "String Indexer"}], "feature": "Country Code", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "266", "mean": "", "stddev": "", "min": "ABW", "max": "ZWE", "missing": "0", "distinct": "266"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country Code"}, {"transformationsData": [{"feature_label": "Indicator Name", "transformation_label": "String Indexer"}], "feature": "Indicator Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "266", "mean": "", "stddev": "", "min": "Children out of school, primary, male", "max": "Children out of school, primary, male", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Indicator Name"}, {"transformationsData": [{"feature_label": "Indicator Code", "transformation_label": "String Indexer"}], "feature": "Indicator Code", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "266", "mean": "", "stddev": "", "min": "SE.PRM.UNER.MA", "max": "SE.PRM.UNER.MA", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Indicator Code"}, {"transformationsData": [{"feature_label": "1960", "transformation_label": "String Indexer"}], "feature": "1960", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1960"}, {"transformationsData": [{"feature_label": "1961", "transformation_label": "String Indexer"}], "feature": "1961", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1961"}, {"transformationsData": [{"feature_label": "1962", "transformation_label": "String Indexer"}], "feature": "1962", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1962"}, {"transformationsData": [{"feature_label": "1963", "transformation_label": "String Indexer"}], "feature": "1963", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1963"}, {"transformationsData": [{"feature_label": "1964", "transformation_label": "String Indexer"}], "feature": "1964", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1964"}, {"transformationsData": [{"feature_label": "1965", "transformation_label": "String Indexer"}], "feature": "1965", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1965"}, {"transformationsData": [{"feature_label": "1966", "transformation_label": "String Indexer"}], "feature": "1966", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1966"}, {"transformationsData": [{"feature_label": "1967", "transformation_label": "String Indexer"}], "feature": "1967", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1967"}, {"transformationsData": [{"feature_label": "1968", "transformation_label": "String Indexer"}], "feature": "1968", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1968"}, {"transformationsData": [{"feature_label": "1969", "transformation_label": "String Indexer"}], "feature": "1969", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "266", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "1969"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1970", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "14", "mean": "233611.57", "stddev": "464272.25", "min": "855", "max": "1728526", "missing": "252"}, "updatedLabel": "1970"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1971", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "41", "mean": "435967.12", "stddev": "1666444.87", "min": "616", "max": "10552260", "missing": "225"}, "updatedLabel": "1971"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1972", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "43", "mean": "130071.28", "stddev": "358855.35", "min": "206", "max": "2288942", "missing": "223"}, "updatedLabel": "1972"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1973", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "131086.52", "stddev": "196233.56", "min": "601", "max": "850115", "missing": "224"}, "updatedLabel": "1973"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1974", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "171979.62", "stddev": "385841.84", "min": "616", "max": "2272313", "missing": "224"}, "updatedLabel": "1974"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1975", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "39", "mean": "135681.26", "stddev": "377584.94", "min": "135", "max": "2330718", "missing": "227"}, "updatedLabel": "1975"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1976", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "45", "mean": "105456.73", "stddev": "320380.95", "min": "431", "max": "2103980", "missing": "221"}, "updatedLabel": "1976"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1977", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "45", "mean": "90354.82", "stddev": "148098.82", "min": "494", "max": "732722", "missing": "221"}, "updatedLabel": "1977"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1978", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "55", "mean": "131784.11", "stddev": "233050.56", "min": "261", "max": "1126115", "missing": "211"}, "updatedLabel": "1978"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1979", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "57", "mean": "90930.0", "stddev": "157241.18", "min": "26", "max": "807615", "missing": "209"}, "updatedLabel": "1979"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1980", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "46", "mean": "74081.98", "stddev": "112888.91", "min": "16", "max": "433159", "missing": "220"}, "updatedLabel": "1980"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1981", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "61", "mean": "125323.87", "stddev": "258158.42", "min": "17", "max": "1616301", "missing": "205"}, "updatedLabel": "1981"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1982", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "61", "mean": "98121.11", "stddev": "171438.64", "min": "301", "max": "805590", "missing": "205"}, "updatedLabel": "1982"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1983", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "65", "mean": "117234.34", "stddev": "242485.59", "min": "131", "max": "1635756", "missing": "201"}, "updatedLabel": "1983"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1984", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "64", "mean": "103191.98", "stddev": "266247.68", "min": "29", "max": "2006002", "missing": "202"}, "updatedLabel": "1984"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1985", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "59", "mean": "122151.85", "stddev": "311938.24", "min": "17", "max": "2136775", "missing": "207"}, "updatedLabel": "1985"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1986", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "64", "mean": "149588.61", "stddev": "329247.34", "min": "32", "max": "2248048", "missing": "202"}, "updatedLabel": "1986"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1987", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "55", "mean": "179992.04", "stddev": "455894.33", "min": "325", "max": "2360441", "missing": "211"}, "updatedLabel": "1987"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1988", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "57", "mean": "189997.21", "stddev": "426547.62", "min": "68", "max": "2301271", "missing": "209"}, "updatedLabel": "1988"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1989", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "57", "mean": "197810.81", "stddev": "458179.69", "min": "156", "max": "2477239", "missing": "209"}, "updatedLabel": "1989"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1990", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "316555.26", "stddev": "1008072.56", "min": "12", "max": "6510760", "missing": "213"}, "updatedLabel": "1990"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1991", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "48", "mean": "244200.71", "stddev": "588500.91", "min": "367", "max": "3739816", "missing": "218"}, "updatedLabel": "1991"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1992", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "49", "mean": "235205.47", "stddev": "694947.14", "min": "310", "max": "4595033", "missing": "217"}, "updatedLabel": "1992"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1993", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "50", "mean": "248306.2", "stddev": "738492.93", "min": "214", "max": "4961843", "missing": "216"}, "updatedLabel": "1993"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1994", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "56", "mean": "298522.77", "stddev": "853842.78", "min": "85", "max": "5260221", "missing": "210"}, "updatedLabel": "1994"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1995", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "55", "mean": "300516.95", "stddev": "820698.09", "min": "175", "max": "4913210", "missing": "211"}, "updatedLabel": "1995"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1996", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "59", "mean": "237003.2", "stddev": "769185.16", "min": "20", "max": "4735287", "missing": "207"}, "updatedLabel": "1996"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1997", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "53", "mean": "210176.62", "stddev": "629437.93", "min": "25", "max": "4335628", "missing": "213"}, "updatedLabel": "1997"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1998", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "50", "mean": "222507.64", "stddev": "527655.95", "min": "119", "max": "3283751", "missing": "216"}, "updatedLabel": "1998"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "1999", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "83", "mean": "270777.11", "stddev": "586151.77", "min": "54", "max": "3343652", "missing": "183"}, "updatedLabel": "1999"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2000", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "93", "mean": "262626.85", "stddev": "796786.17", "min": "24", "max": "6301261", "missing": "173"}, "updatedLabel": "2000"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2001", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "82", "mean": "258137.79", "stddev": "867766.83", "min": "482", "max": "7080068", "missing": "184"}, "updatedLabel": "2001"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2002", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "88", "mean": "249389.07", "stddev": "904341.27", "min": "18", "max": "7830397", "missing": "178"}, "updatedLabel": "2002"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2003", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "88", "mean": "239727.89", "stddev": "880828.57", "min": "35", "max": "7612634", "missing": "178"}, "updatedLabel": "2003"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2004", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "84", "mean": "179600.7", "stddev": "494476.87", "min": "11", "max": "3161902", "missing": "182"}, "updatedLabel": "2004"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2005", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "95", "mean": "150671.51", "stddev": "430820.96", "min": "14", "max": "3141676", "missing": "171"}, "updatedLabel": "2005"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2006", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "89", "mean": "146013.31", "stddev": "438787.19", "min": "38", "max": "3276269", "missing": "177"}, "updatedLabel": "2006"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2007", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91", "mean": "146457.01", "stddev": "458851.67", "min": "13", "max": "2798835", "missing": "175"}, "updatedLabel": "2007"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2008", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "95", "mean": "167784.95", "stddev": "566117.29", "min": "51", "max": "3753004", "missing": "171"}, "updatedLabel": "2008"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2009", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "95", "mean": "186324.37", "stddev": "666934.27", "min": "85", "max": "5033204", "missing": "171"}, "updatedLabel": "2009"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2010", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91", "mean": "178103.07", "stddev": "649634.23", "min": "174", "max": "4463361", "missing": "175"}, "updatedLabel": "2010"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "96", "mean": "82839.29", "stddev": "209584.83", "min": "34", "max": "1622712", "missing": "170"}, "updatedLabel": "2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "110", "mean": "118926.55", "stddev": "377320.07", "min": "29", "max": "3267567", "missing": "156"}, "updatedLabel": "2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "99", "mean": "100568.76", "stddev": "271686.95", "min": "31", "max": "2117484", "missing": "167"}, "updatedLabel": "2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "106", "mean": "95921.58", "stddev": "219558.12", "min": "13", "max": "1250835", "missing": "160"}, "updatedLabel": "2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "106", "mean": "104219.06", "stddev": "236900.24", "min": "21", "max": "1291133", "missing": "160"}, "updatedLabel": "2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "108", "mean": "147949.42", "stddev": "582772.53", "min": "82", "max": "5785719", "missing": "158"}, "updatedLabel": "2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2017", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "104", "mean": "96748.9", "stddev": "195219.49", "min": "120", "max": "1203156", "missing": "162"}, "updatedLabel": "2017"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2018", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105", "mean": "160301.66", "stddev": "704010.86", "min": "41", "max": "7000986", "missing": "161"}, "updatedLabel": "2018"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2019", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105", "mean": "142640.28", "stddev": "740976.62", "min": "67", "max": "7500381", "missing": "161"}, "updatedLabel": "2019"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2020", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105", "mean": "116221.97", "stddev": "527878.46", "min": "21", "max": "5257913", "missing": "161"}, "updatedLabel": "2020"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2021", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "101", "mean": "175940.66", "stddev": "543244.95", "min": "17", "max": "3709281", "missing": "165"}, "updatedLabel": "2021"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2022", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "46", "mean": "129737.8", "stddev": "321940.47", "min": "19", "max": "1954473", "missing": "220"}, "updatedLabel": "2022"}]}))

	#transformationPostExecutionHook(schoolpredictionpautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run schoolpredictionpHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(schoolpredictionpautofe, ["1970", "1972", "1973", "1974", "1975", "1976", "1977", "1978", "1979", "1980", "1981", "1982", "1983", "1984", "1985", "1986", "1987", "1988", "1989", "1990", "1991", "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "Country Name_stringindexer", "Country Code_stringindexer", "Indicator Name_stringindexer", "Indicator Code_stringindexer", "1960_stringindexer", "1961_stringindexer", "1962_stringindexer", "1963_stringindexer", "1964_stringindexer", "1965_stringindexer", "1966_stringindexer", "1967_stringindexer", "1968_stringindexer", "1969_stringindexer"], "1971")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

